In [1]:
## Data Preprocessing

import glob
from PIL import Image
import numpy as np

img_rows = 96
img_cols = 96
nb_classes = 22 #23

# Generate X and y for Convolutional Neural Network
X = []
y = []

paths = glob.glob('./cropped_cells/*.jpg')
#np.random.shuffle(paths) #randomly shuffle data set

counter = 0

# Get all the images and convert them into greyscale
for path in paths:
    im = Image.open(path)
    grey_im = im.convert('L')
    # Convert them into numpy
    label = [path.split('_')[1].split('/')[1]] # extract the label from path
    # balance the dataset
    if label == ['interphase'] and counter < 800:
        #counter += 1
        pass
    else:
        y += [path.split('_')[1].split('/')[1]] # extract the label from path
        X += [np.array(grey_im)]
    
X = np.array(X)
print X.shape
y = np.array(y)

X_train = X # stupid, taking full dataset

# filtered interphase numbers
f = filter(lambda x: x == 'interphase', y)
len(f)

(1306, 96, 96)


0

In [2]:
# Normalize 

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
#X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
X_train /= 255 #Normalize
#X_test /= 255
print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')

(1306, 'train samples')


In [3]:
# Convert y into Y

from keras.utils import np_utils, generic_utils

# Convert labels to numeric
y_unique = np.unique(y)
dic = {}

for i, label in enumerate(y_unique):
    dic[label] = i
print dic

y_numeric = []
for el in y:
    y_numeric += [dic[el]]
    
y_numeric # now a 2000 label vector
Y = np_utils.to_categorical(y_numeric, nb_classes)

print Y.shape

Y_train = Y

{'halfcircle': 7, 'metaphase': 14, 'earlyprophase': 4, 'debris': 3, 'nucleolirim': 18, 'kidney': 10, 'telophase': 21, 'blurry': 2, 'fragmented': 6, 'apoptotic': 1, 'latepro': 11, 'multinucleate': 17, 'prophase': 19, 'anaphase': 0, 'indented': 9, 'monopole': 16, 'holey': 8, 'latetelophase': 12, 'lines': 13, 'micronucleus': 15, 'round': 20, 'elongated': 5}
(1306, 22)


In [4]:
from sklearn.cross_validation import StratifiedKFold

#print y_numeric
skl = StratifiedKFold(y_numeric, n_folds=5)


for train,test in skl:
    print len(train), len(test)
    
X_stratified = X_train[train]
Y_stratified = Y[train]

X_stratified = np.append(X_stratified,X_train[test],axis=0)
Y_stratified = np.append(Y_stratified,Y[test],axis=0)

X_stratified.shape
Y_stratified.shape

1036 270
1039 267
1046 260
1049 257
1054 252


(1306, 22)

In [5]:
## Convolutional Neural Network with 2 convolutions

from keras.models import Sequential
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad, Adam
from keras.layers.normalization import BatchNormalization




batch_size = 128
nb_epoch = 12
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3
# the data images are greyscale
img_channels = 1

Using Theano backend.
Couldn't import dot_parser, loading of dot files will not be possible.

Using gpu device 0: GRID K520 (CNMeM is disabled)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/Theano-0.8.0.dev0-py2.7.egg/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [ ]:
model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv, activation='relu',border_mode='valid', input_shape=(img_channels, img_rows, img_cols)))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv, activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv, activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv, activation='relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [7]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd')

In [6]:
## VGG Net inspired

model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(img_channels,img_rows,img_cols)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(nb_classes, activation='softmax'))

# let's train the model using SGD + momentum (how original).
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(X_train)

In [ ]:
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=32),
                    samples_per_epoch=len(X_train), nb_epoch=1000, show_accuracy=True)

In [ ]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=1000,validation_split=0.3, show_accuracy=True)

In [ ]:
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=adam)

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd')

In [8]:
model.fit(X_stratified, Y_stratified, batch_size=batch_size, nb_epoch=1000,validation_split=0.2, show_accuracy=True)

Train on 1044 samples, validate on 262 samples
Epoch 1/1000
1044/1044 [==============================] - 11s - loss: 3.0866 - acc: 0.0603 - val_loss: 3.0764 - val_acc: 0.1069
Epoch 2/1000
1044/1044 [==============================] - 11s - loss: 3.0754 - acc: 0.0642 - val_loss: 3.0648 - val_acc: 0.1145
Epoch 3/1000
1044/1044 [==============================] - 11s - loss: 3.0693 - acc: 0.0642 - val_loss: 3.0499 - val_acc: 0.1145
Epoch 4/1000
1044/1044 [==============================] - 11s - loss: 3.0506 - acc: 0.0747 - val_loss: 3.0284 - val_acc: 0.1145
Epoch 5/1000
1044/1044 [==============================] - 11s - loss: 3.0541 - acc: 0.0757 - val_loss: 3.0189 - val_acc: 0.1145
Epoch 6/1000
1044/1044 [==============================] - 11s - loss: 3.0350 - acc: 0.0805 - val_loss: 2.9837 - val_acc: 0.1145
Epoch 7/1000
1044/1044 [==============================] - 11s - loss: 3.0274 - acc: 0.0929 - val_loss: 2.9845 - val_acc: 0.1145
Epoch 8/1000
1044/1044 [==============================] -